# **Validation of a Mitochondrial Polygenic Score (MGS) for Parkinson's Disease - iPD European Ancestry**

## Project Title: Validation of a Mitochondrial Polygenic Score (MGS) for Parkinson's Disease

**V:** GATK 4.3.0.0, Python 3.10.12, R 4.4.2

**Note:** This notebook is for the European (EUR) ancestry group, to apply to other ancestry groups simply change the "EUR" to one of the following ancestries:

* African Admixed (AAC)
* African (AFR)
* Ashkenazi Jewish (AJ)
* American Admixed (AMR)
* Central Asian (CAS)
* East Asian (EAS)
* European (EUR)
* Middle Eastern (MDE)
* South Asian (SAS)

## Description:

- [1. Getting started](#getting-started)
- [2. Copying data to workspace](#copying-data-to-workplace)
- [3. Formatting and QC](#formatting-and-qc)
- [4. Calculating MGS](#calculating-mgs)
- [5. Statistical Analyses](#statistical-analyses)
- [6. Statistical Analyses on iPD group](#statistical-analyses-on-ipd-group)
- [7. Saving results](#saving-results)


For more information contact Joshua Ooi

Last updated: 06/02/2025

# Getting Started

## Load Python Libraries

In [ ]:
# Use the os package to interact with the environment (helps me find the proper paths to things)
import os
import sys

# Bring in Pandas for Dataframe functionality (popular python packages to read in my data, manipulate, subset, filter, etc.)
import pandas as pd
from functools import reduce

# Bring some visualization functionality (visualisation package useful when plotting stuff)
import seaborn as sns

# numpy for basics (mathematics package)
import numpy as np

# Use StringIO for working with file contents (so that it's interacting with terra cloud)
from io import StringIO

# Enable IPython to display matplotlib graphs (also a visualisation package)
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API (needs to be enabled in order to have terra interacting with data in the buckets)
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console (just to display things in a jupyter notebook)
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console (for interactions between terra and the cloud)
import urllib.parse

# BigQuery for querying data (for interactions between terra and the cloud)
from google.cloud import bigquery

print('Buenos Dias, Joshua!')

## Define Python Functions to Interact with GCP/Terra

In [ ]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command

def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)

def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

## Initialise Work Environment Variables

In [ ]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Accessing GP2 Data

In [ ]:
##  GP2 v6.0
GP2_RELEASE_PATH = '/GP2/release/path' ##  Enter valid GP2 Release path
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_META_RELEASE_PATH = f'{GP2_RELEASE_PATH}/meta_data'
GP2_SUMSTAT_RELEASE_PATH = f'{GP2_RELEASE_PATH}/summary_statistics'
GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
GP2_WGS_PATH = f'{GP2_RELEASE_PATH}/wgs'
print('GP2 v6.0')
print(f'Path to GP2 v6.0 Clinical Data: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v6.0 Raw Genotype Data: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v6.0 Imputed Genotype Data: {GP2_IMPUTED_GENO_PATH}')
print(f'Path to GP2 v6.0 Metadata: {GP2_META_RELEASE_PATH}')
print(f'Path to GP2 v6.0 WGS Data: {GP2_WGS_PATH}')

## Install PLINK

In [ ]:
%%capture
%%bash
# Install plink 1.9
cd /home/jupyter/
if test -e /home/jupyter/plink; then

echo "Plink is already installed in /home/jupyter/"
else
echo "Plink is not installed"
cd /home/jupyter

wget http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip

unzip -o plink_linux_x86_64_20190304.zip
mv plink plink1.9

fi

In [ ]:
%%capture
%%bash
# Install plink 2.0
cd /home/jupyter/
if test -e /home/jupyter/plink2; then

echo "Plink2 is already installed in /home/jupyter/"
else
echo "Plink2 is not installed"
cd /home/jupyter/

wget http://s3.amazonaws.com/plink2-assets/plink2_linux_x86_64_latest.zip

unzip -o plink2_linux_x86_64_latest.zip

fi

In [ ]:
%%bash
# chmod plink 1.9 to make sure I have permission to run the program
chmod u+x /home/jupyter/plink1.9

In [ ]:
%%bash
# chmod plink 2.0 to make sure I have permission to run the program
chmod u+x /home/jupyter/plink2

## Install R

In [ ]:
# Install R
! pip install --upgrade rpy2

In [ ]:
pip install --upgrade pip

In [ ]:
%load_ext rpy2.ipython

# Copying Data to Workspace

## Make a Folder

In [ ]:
# Assign ancestry
ANCESTRY = "EUR"
os.environ["ANCESTRY"] = ANCESTRY

In [ ]:
# Create a folder on my workspace
print("Making a working directory")
WORK_DIR = f'/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO'
shell_do(f'mkdir -p {WORK_DIR}')
os.environ["WORK_DIR"] = WORK_DIR

## Copy Files from GP2 Buckets Over to My Folder

In [ ]:
# Copy files from GP2 bucket to my workspace - Focus on the EUR
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_IMPUTED_GENO_PATH}/{ANCESTRY} {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_CLINICAL_RELEASE_PATH}/master_key_release6_final.csv {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_META_RELEASE_PATH}/related_samples/release6_{ANCESTRY}.related {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_RAW_GENO_PATH}/{ANCESTRY}/{ANCESTRY}_release6.eigenvec {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp {GP2_RAW_GENO_PATH}/{ANCESTRY}/{ANCESTRY}_release6.eigenval {WORK_DIR}

## Copy non-GP2 Bucket Files Over to My Folder

In [ ]:
# Copy files containing MGS SNPs and weights
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp /enter/relevant/path/MGS_HG38.profile.txt {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp /enter/relevant/path/MGS_HG38.profile.raw {WORK_DIR}')

# Formatting and QC

## Convert pvar files to bim

In [ ]:
%%bash -s "$WORK_DIR"
cd $1

for chnum in {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22};
do
    /home/jupyter/plink2 \
    --pfile ${ANCESTRY}/chr"$chnum"_${ANCESTRY}_release6 \
    --make-bed \
    --out ${ANCESTRY}/MGS_chr"$chnum"_${ANCESTRY}_release6
done

## QC *(minor allele frequency >0.01, missingness per SNP<0.05, missingness per sample <0.02 and Hardy-Weinberg equilibrium >1 × 10−50)*

In [ ]:
%%bash

WORK_DIR=/home/jupyter/WD_GP2_MITO_AIM1_${ANCESTRY}_PD_JO
cd $WORK_DIR

for chnum in {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22};
do
    /home/jupyter/plink2 \
    --bfile ${ANCESTRY}/MGS_chr"$chnum"_${ANCESTRY}_release6 \
    --allow-no-sex \
    --maf 0.01 \
    --geno 0.05 \
    --mind 0.02 \
    --hwe 1e-50 \
    --make-bed \
    --out ${ANCESTRY}/MGS_chr"$chnum"_${ANCESTRY}_release6_QC
done

## Concatenate the bim files to look the same as the MGS_HG38.txt file

In [ ]:
%%bash -s "$WORK_DIR" "$ANCESTRY"
cd $1/$2

for chnum in {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22};
do
    awk '{print $1, $1":"$4, $3, $4, $5, $6}' MGS_chr"$chnum"_${ANCESTRY}_release6_QC.bim > temp
    mv temp MGS_chr"$chnum"_${ANCESTRY}_release6_QC.bim
done

# Calculating MGS

## Extract Risk SNPs

In [ ]:
%%bash -s "$WORK_DIR"
cd $1

for chnum in {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22};
do
    /home/jupyter/plink2 \
    --bfile ${ANCESTRY}/MGS_chr"$chnum"_${ANCESTRY}_release6_QC \
    --extract MGS_HG38.profile.txt \
    --make-bed \
    --out ${ANCESTRY}/MGS_chr"$chnum"_${ANCESTRY}_release6_distilled
done

## Merge distilled files into a single file

In [ ]:
%%bash -s "$WORK_DIR" "$ANCESTRY"
cd $1/$2

# Make a list that includes the files per chr that I just generated
ls *distilled.bim > temp
head temp

In [ ]:
%%bash -s "$WORK_DIR" "$ANCESTRY"
cd $1/$2

# Remove *bim so i only keep the core name
sed 's/.bim//g' temp > temp2
cat temp2

In [ ]:
%%bash -s "$WORK_DIR" "$ANCESTRY"
cd $1/$2

# Remove chr1 from the list so I do not merge it twice
grep -v "MGS_chr1_${ANCESTRY}_release6_distilled" temp2 > files.txt
cat files.txt

In [ ]:
%%bash -s "$WORK_DIR" "$ANCESTRY"
cd $1/$2

# Merge list using chr 1 as my core input
/home/jupyter/plink1.9 \
--bfile MGS_chr1_${ANCESTRY}_release6_distilled \
--merge-list files.txt \
--make-bed \
--out MGS_ALL_${ANCESTRY}_release6

## Calculate MGS score proper

In [ ]:
%%bash -s "$WORK_DIR"
cd $1

/home/jupyter/plink1.9 \
--bfile ${ANCESTRY}/MGS_ALL_${ANCESTRY}_release6 \
--score MGS_HG38.profile.txt \
--out ${ANCESTRY}/MGS_ALL_${ANCESTRY}_release6_score

# Statistical Analyses

####  **Reformat Covariate Files**

In [ ]:
WORK_DIR = f'/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO'
FULL_PATH = WORK_DIR + '/master_key_release6_final.csv'
cov = pd.read_csv(FULL_PATH)
cov.columns

In [ ]:
cov_reduced = cov[['GP2sampleID', 'sex_for_qc', 'age','age_of_onset']]
cov_reduced.rename(columns = {'GP2sampleID':'IID'}, inplace = True)

####  **Merge with Eigenvec Files**

In [ ]:
WORK_DIR = f'/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO'
FULL_PATH = WORK_DIR + f'/{ANCESTRY}_release6.eigenvec'
cov2 = pd.read_csv(FULL_PATH)
cov2.columns

In [ ]:
WORK_DIR = f'/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO'
FULL_PATH = WORK_DIR + f'/{ANCESTRY}_release6.eigenvec'
cov2 = pd.read_csv(FULL_PATH)
cov2.head()
cov_final = cov2.merge(cov_reduced, on='IID') # Horizontal.
cov_final.to_csv(f'/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO/{ANCESTRY}/gp2_covs.csv', header=True, index=False)

## Preliminary Regression Analyses (blind to idiopathic or monogenic status)

In [ ]:
%%R
ANCESTRY <- "EUR"

In [ ]:
%%R

# Load the glue package
library(glue)
ANCESTRY <- Sys.getenv("ANCESTRY")

work_dir <- glue("/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO")
pack <- work_dir

profile <- glue("/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO/{ANCESTRY}/MGS_ALL_{ANCESTRY}_release6_score.profile")
covar <- glue("/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO/{ANCESTRY}/gp2_covs.csv")

temp_data <- read.table(profile, header = T)
temp_covs <- read.csv(covar, header = T, sep=",")
data <- merge(temp_data, temp_covs, by = "IID")
data$CASE <- data$PHENO - 1
data$sex_for_qc <- as.numeric(data$sex_for_qc)
meanControls <- mean(data$SCORE[data$CASE == 0])
sdControls <- sd(data$SCORE[data$CASE == 0])
data$zSCORE <- (data$SCORE - meanControls)/sdControls

In [ ]:
%%R

table(data$CASE)

In [ ]:
%%R

data <- data[data$CASE != -10, ]

In [ ]:
%%R

table(data$CASE)

####   **Regression against disease status**

In [ ]:
%%R
grsTests <- glm(CASE ~ zSCORE + sex_for_qc + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + age, family="binomial", data = data)
summary(grsTests)

# Extract beta and SE from the linear regression model
beta <- coef(grsTests)["zSCORE"]
SE <- summary(grsTests)$coefficients["zSCORE", "Std. Error"]

# Calculate OR, U95, and L95
OR <- exp(beta)
U95 <- exp((beta) + (1.96 * SE))
L95 <- exp((beta) - (1.96 * SE))

# Print results
print(summary(grsTests))

# Print results
print(OR)
print(L95)
print(U95)

####   **Multivariable linear regression against AAO**

In [ ]:
%%R
cases <- subset(data, CASE == 1)
meanPop <- mean(cases$SCORE)
sdPop <- sd(cases$SCORE)
cases$zSCORE <- (cases$SCORE - meanPop)/sdPop
grsTests <- lm(age_of_onset ~ zSCORE + sex_for_qc + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data = cases)
summary(grsTests)

# Statistical Analyses on iPD Group

## Establishing the iPD group

####   **Bring in the csv files containing the carriers of *LRRK2, PINK, PRKN* mutations**

In [ ]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp /enter/relevant/path/LRRK2_carriers_all.csv {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp /enter/relevant/path/pp_hombil.csv {WORK_DIR}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp /enter/relevant/path/pp_hombilhet.csv {WORK_DIR}')

####   **Start by creating a new column labelling those with pp hombil (meaning pathogenic *PRKN/PINK1* patients) as carriers**

In [ ]:
WORK_DIR = f'/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO'
FULL_PATH = WORK_DIR + '/pp_hombil.csv'

# Load the main CSV file containing subject IDs and MGS scores
main_df = pd.read_csv(f'/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO/{ANCESTRY}/gp2_covs.csv')

# Load the CSV file containing the list of carrier subject IDs
carrier_df = pd.read_csv(FULL_PATH)

# Create a new column in the main dataframe to indicate carrier status
main_df["Carrier"] = 0

# Iterate over each subject ID in the carrier list
for subject_id in carrier_df['IID']:
    # Mark the corresponding rows in the main dataframe as carriers
    main_df.loc[main_df['IID'] == subject_id, "Carrier"] = 1

# Save the updated dataframe to a new CSV file
main_df.to_csv(f'/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO/{ANCESTRY}/updated_pphombil_gp2_covs.csv', index=False)

####   **Then add the *LRRK2* carriers to that carrier status column too**

In [ ]:
# Load the updated main CSV file containing subject IDs, MGS scores, and the carrier column
main_df = pd.read_csv(f'/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO/{ANCESTRY}/updated_pphombil_gp2_covs.csv')

# Load the new CSV file containing the list of additional carrier subject IDs
additional_carrier_df = pd.read_csv(f'/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO/LRRK2_carriers_all.csv', delimiter='\t')
for subject_id in additional_carrier_df['IID']:
    # Check if the subject ID is already in the main dataframe
    if subject_id in main_df['IID'].values:
        # Mark the corresponding rows as carriers without changing the designation for existing carriers
        main_df.loc[main_df['IID'] == subject_id, "Carrier"] = 1

# Save the updated dataframe to a new CSV file
main_df.to_csv(f'/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO/{ANCESTRY}/updated_lrrk2pphombil_gp2_covs.csv', index=False)

####   **Some data formatting**

In [ ]:
%%R -i ANCESTRY

# Load the glue package
# install.packages("glue")
library(glue)

# Pass ancestry variable
work_dir <- glue("/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO")
pack <- work_dir
profile <- glue("/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO/{ANCESTRY}/MGS_ALL_{ANCESTRY}_release6_score.profile")
covar <- glue("/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO/{ANCESTRY}/updated_lrrk2pphombil_gp2_covs.csv")
temp_data <- read.table(profile, header = T)
temp_covs <- read.csv(covar, header = T, sep=",")
data <- merge(temp_data, temp_covs, by = "IID")
data$CASE <- data$PHENO - 1
data$sex_for_qc <- as.numeric(data$sex_for_qc)
meanControls <- mean(data$SCORE[data$CASE == 0])
sdControls <- sd(data$SCORE[data$CASE == 0])
data$zSCORE <- (data$SCORE - meanControls)/sdControls

In [ ]:
%%R

table(data$CASE)

In [ ]:
%%R

data <- data[data$CASE != -10, ]

In [ ]:
%%R

table(data$CASE)

## Comparing between cases and controls

####   **Count means and SDs of standardised scores**

In [ ]:
%%R

library(dplyr)

# Calculate mean and SD for standardised scores
summary_stats <- data %>%
  group_by(CASE) %>%
  summarize(
    mean_standardized = mean(zSCORE, na.rm = TRUE),
    sd_standardized = sd(zSCORE, na.rm = TRUE)
  ) %>%
  mutate(CASE = ifelse(CASE == 0, "Controls", "Cases")) %>%
  rename(Group = CASE)

# Print the summary statistics
print(summary_stats)

####   **Deciding which statistical test to use to compare means**

####   *Check for normality of control data*

In [ ]:
%%R

library(ggplot2)

# Histogram for Controls (CASE == 0)
ggplot(data[data$CASE == 0, ], aes(x = zSCORE)) +
  geom_histogram(binwidth = 0.2, fill = "blue", alpha = 0.7) +
  ggtitle("Histogram of zSCORE for Controls")

In [ ]:
%%R

library(ggplot2)

# Q-Q plots to check normality for controls
qqnorm(data$zSCORE[data$CASE == 0], main = "Q-Q Plot for Controls")
qqline(data$zSCORE[data$CASE == 0])

####   *Check for normality of cases data*

In [ ]:
%%R

library(ggplot2)

# Histogram for Cases (CASE == 1)
ggplot(data[data$CASE == 1, ], aes(x = zSCORE)) +
  geom_histogram(binwidth = 0.2, fill = "red", alpha = 0.7) +
  ggtitle("Histogram of zSCORE for Cases")

In [ ]:
%%R

library(ggplot2)

# Q-Q plots to check normality for cases
qqnorm(data$zSCORE[data$CASE == 1], main = "Q-Q Plot for Cases")
qqline(data$zSCORE[data$CASE == 1])

####   *Check for equal variances*

In [ ]:
%%R
install.packages("car")

####   *Check for homogeniety of variances*

In [ ]:
%%R

data$CASE <- as.factor(data$CASE)

# Run Levene's test
library(car)
leveneTest(zSCORE ~ CASE, data = data)

## Comparisons

####   **T-test or Mann–Whitney**

####   *If data normally distributed and has no outliers*

In [ ]:
%%R
t.test(zSCORE ~ CASE, data = data, var.equal = TRUE)

####   *If data not normally distributed or has outliers*

In [ ]:
%%R
wilcox.test(zSCORE ~ CASE, data = data)

## Visualisations

####   ***I quite like wesanderson colours***

In [ ]:
%%R
install.packages("wesanderson")

In [ ]:
%%R
library(wesanderson)

####   ***Violin plots***

In [ ]:
%%R -i ANCESTRY

library(ggplot2)
library(wesanderson)
library(glue)

colors <- wes_palette(name = "Chevalier1", n = 2)

swapped_colors <- colors[c(2, 1)]

# Create a violin plot
p <- ggplot(data, aes(x = reorder(as.factor(CASE), zSCORE), y = zSCORE, fill = as.factor(CASE))) +
  geom_violin(trim = FALSE) +
  geom_boxplot(width = 0.4, fill = "white") +
  theme_minimal() +
  scale_fill_manual(values = swapped_colors) +
  theme_bw() +
  ylab("Standardized MGS") +
  xlab("") +
  theme(legend.position = "none")

# Save the plot as a JPEG file
output_jpeg <- glue("/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO/{ANCESTRY}_violin.jpeg")
ggsave(output_jpeg, dpi = 600, units = "in", height = 6, width = 6)

# Display the plot
p

####   ***Density plots***

In [ ]:
%%R -i ANCESTRY
# Density plot - probably best visualised without wesanderson colours
library(ggplot2)
library(glue)

# Pass ancestry variable
work_dir <- glue("/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO")
pack <- work_dir
profile <- glue("/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO/{ANCESTRY}/MGS_ALL_{ANCESTRY}_release6_score.profile")
covar <- glue("/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO/{ANCESTRY}/updated_lrrk2pphombil_gp2_covs.csv")
temp_data <- read.table(profile, header = T)
temp_covs <- read.csv(covar, header = T, sep=",")
data <- merge(temp_data, temp_covs, by = "IID")
data$CASE <- data$PHENO - 1
data <- data[data$CASE != -10, ]
meanControls <- mean(data$SCORE[data$CASE == 0])
sdControls <- sd(data$SCORE[data$CASE == 0])
head(data)
data$zSCORE <- (data$SCORE - meanControls)/sdControls

Model <- glm(CASE ~ SCORE, data = data, family = 'binomial')
data$probDisease <- predict(Model, data, type = c("response"))
data$predicted <- ifelse(data$probDisease > 0.5, "DISEASE", "CONTROL")
data$reported <- ifelse(data$CASE == 1, "DISEASE","CONTROL")

# Density plot
output_png <- glue("/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO/{ANCESTRY}_density.png")
densPlot <- ggplot(data, aes(probDisease, fill = reported, color = reported)) + geom_density(alpha = 0.5) + theme_bw()
ggsave(plot = densPlot, filename = output_png, width = 8, height = 5, units = "in", dpi = 300)
densPlot

## AUC analyses

In [ ]:
%%R
install.packages("pROC")

In [ ]:
%%R
library(pROC)

In [ ]:
%%R

library(pROC)

Model <- glm(CASE ~ SCORE, data = data, family = 'binomial')

data$probDisease <- predict(Model, data, type = "response")

roc_curve <- roc(data$CASE, data$probDisease)

# print AUC value
auc_value <- auc(roc_curve)
cat("AUC Value:", auc_value, "\n")

# plot ROC curve
plot(roc_curve, main = "ROC Curve", col = "blue")

In [ ]:
%%R

# Extract the confidence interval for the AUC
auc_ci <- ci(roc_curve)

# Print AUC and CI
cat("AUC:", auc(roc_curve), "\n")
cat("95% CI for AUC:", auc_ci[1], "-", auc_ci[3], "\n")

# Check if the CI includes 0.5
if (auc_ci[1] > 0.5 | auc_ci[3] < 0.5) {
  cat("The AUC is significantly different from 0.5 at the 95% confidence level.\n")
} else {
  cat("The AUC is not significantly different from 0.5 at the 95% confidence level.\n")
}

In [ ]:
%%R

# Extract scores for positive and negative classes
positive_scores <- roc_curve$predictor[roc_curve$response == 1]
negative_scores <- roc_curve$predictor[roc_curve$response == 0]

# Perform a Wilcoxon rank-sum test
wilcox_test <- wilcox.test(positive_scores, negative_scores, alternative = "greater")
cat("P-value from Wilcoxon test:", wilcox_test$p.value, "\n")

# Multivariable logistic regression

####   **Regression against disease status for those who are not carriers of mutations**

####   **Correct for age, sex, and PCs**

In [ ]:
%%R

non_carrier_subset <- subset(data, Carrier == 0)

# Perform logistic regression
grsTests <- glm(CASE ~ zSCORE + sex_for_qc + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + age, family = binomial, data = non_carrier_subset)
summary(grsTests)

# Extract beta and SE from the linear regression model
beta <- coef(grsTests)["zSCORE"]
SE <- summary(grsTests)$coefficients["zSCORE", "Std. Error"]

# Calculate OR, U95, and L95
OR <- exp(beta)
U95 <- exp((beta) + (1.96 * SE))
L95 <- exp((beta) - (1.96 * SE))

# Print results
print(summary(grsTests))

# Print results
print(OR)
print(L95)
print(U95)

## Multivariable linear regression

####   **Regression against AAO with those who are not carriers of mutations in either genes (i.e. iPD patients)**

In [ ]:
%%R
subset_cases <- subset(data, CASE == 1 & (is.na(Carrier) | Carrier == 0))

# Calculate the mean and sd based on the subset
meanPop <- mean(subset_cases$SCORE)
sdPop <- sd(subset_cases$SCORE)

# Calculate z-scores within the same subset
subset_cases$zSCORE <- (subset_cases$SCORE - meanPop) / sdPop

# Perform the linear regression on the same subset
grsTests <- lm(age_of_onset ~ zSCORE + sex_for_qc + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10, data = subset_cases)
summary(grsTests)

In [ ]:
%%R
# Assuming zSCORE and age_of_onset are in subset_cases

# Calculate the correlation
cor_test <- cor.test(subset_cases$zSCORE, subset_cases$age_of_onset)
cor_coefficient <- cor_test$estimate
p_value <- cor_test$p.value

# Print the results
cat("Correlation coefficient (r):", cor_coefficient, "\n")
cat("p-value:", p_value, "\n")

# Create a correlation plot
library(ggplot2)

# Generate the correlation plot
cor_plot <- ggplot(subset_cases, aes(x = zSCORE, y = age_of_onset)) +
  geom_point(alpha = 0.6) +
  geom_smooth(method = "lm", se = FALSE, color = "red") +
  theme_minimal() +
  xlab("Z-Score") +
  ylab("Age of Onset") +
  ggtitle(paste("Correlation: r =", round(cor_coefficient, 2), ", p =", round(p_value, 3)))

# Display the plot
print(cor_plot)

# Optionally save the plot
output_jpeg <- glue("/home/jupyter/WD_GP2_MITO_AIM1_{ANCESTRY}_PD_JO/{ANCESTRY}_corr.jpeg")
ggsave(output_jpeg, plot = cor_plot, dpi = 600, units = "in", height = 6, width = 6)

# Saving

Save the final files to your workspace bucket, since we are conducting this analysis on Terra.